In [1]:
import pandas as pd
import numpy as np
import os
import gc
import datetime
import gc
import logging

today_str=str(datetime.datetime.now().date())
logging.basicConfig(filename='control_test_'+today_str+'_.log', level=logging.INFO)

In [2]:
today_str=str(datetime.datetime.now().date())

In [3]:
# The comparing pool are thoses stores only have sales every week before the beginning of 2018 Q1 campaign

sales_base_week_list=[datetime.datetime(2018,1,20).date(),datetime.datetime(2018,1,27).date(),datetime.datetime(2018,2,3).date()] # start of Jan.14, 3 weeks in total

base1_week=datetime.datetime(2018,2,3).date()
campaign1_week=datetime.datetime(2018,2,10).date()

base2_week=datetime.datetime(2018,3,24).date()
campaign2_week=datetime.datetime(2018,3,31).date()


Q1_start_week=datetime.datetime(2018,2,10) # start of Feb.4
Q1_end_week=datetime.datetime(2018,5,5)

In [4]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
store_zips=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','zip']]
store_zips['zip']=store_zips['zip'].apply(lambda x: x.zfill(5))
inclusion_stores=pd.merge(inclusion_stores,store_zips,on="location_id",how="left")
inclusion_stores=inclusion_stores[['location_id','zip']].rename(columns={"zip":"zip_cd"})


In [5]:
sales_all=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-06-02.csv",dtype=str)

In [6]:
sales_all=sales_all[['location_id','week_end_date','sales']]
sales_all['sales']=sales_all['sales'].astype(float)
sales_all['week_end_date']=sales_all['week_end_date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_inclusion_long=sales_all[sales_all['location_id'].isin(inclusion_stores['location_id'])]

store_DMA=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','DMA']]

In [7]:

# 1. Recent 52 weeks sales
sales_all=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-06-02.csv",dtype=str)
sales_all['week_end_date']=sales_all['week_end_date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_all['sales']=sales_all['sales'].astype(float)
sales_52_weeks=sales_all[(sales_all['week_end_date']>=datetime.datetime(2017,2,1).date()) & (sales_all['week_end_date']<=datetime.datetime(2017,7,31).date())]
sales_52_weeks['sales']=sales_52_weeks['sales'].astype(float)
sales_52_weeks['transactions']=sales_52_weeks['transactions'].astype(float)
sales_52_weeks['traffics']=sales_52_weeks['traffics'].astype(float)
del sales_52_weeks['week_indicator']

df_sales_52=sales_52_weeks[['location_id','week_end_date','sales']]
df_sales_52=df_sales_52[df_sales_52['sales']!=0]
df_sales_52_dollar=df_sales_52.groupby("location_id")['sales'].sum().to_frame().reset_index().rename(columns={"sales":"52_week_sales"})
df_sales_52_week=df_sales_52.groupby("location_id")['week_end_date'].count().to_frame().reset_index().rename(columns={"week_end_date":"week_count"})
df_sales_52=pd.merge(df_sales_52_dollar,df_sales_52_week,on="location_id",how="inner")
df_sales_52=df_sales_52[df_sales_52['week_count']==df_sales_52['week_count'].max()]

Df_parameters=pd.merge(df_sales_52,store_DMA,on="location_id",how="left")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [8]:
df_sales_52['week_count'].max()

26

In [9]:
# 2. Jan Sales
# No extra exclusion for the Jan weeks involoved
sales_all_base_Jan=sales_inclusion_long[sales_inclusion_long['week_end_date'].isin(sales_base_week_list)]
sales_all_base_Jan=sales_all_base_Jan.groupby(['location_id'])['sales'].sum().to_frame().reset_index()
sales_all_base_Jan=sales_all_base_Jan.rename(columns={"sales":"Jan_3_weeks_sales"})

In [10]:
Df_parameters=pd.merge(Df_parameters,sales_all_base_Jan,on="location_id",how="left")

In [11]:
sales_all.head(2)

,location_id,week_end_date,sales,transactions,traffics,week_indicator
0,1,2015-05-23,84370.89,2823.0,4631,1
1,1,2015-05-30,85665.43,2945.0,4873,2


In [12]:
#3. ARI
def ARI_calculation(df_all,base_week,campaign_week):
    df=df_all[df_all['week_end_date'].isin(base_week+campaign_week)]

    df_0=df[df['sales']==0]
    df=df[~df['location_id'].isin(df_0)]
    df=df[df['sales']>0]
    
    df_both_weeks=df.groupby("location_id")['week_end_date'].count().to_frame().reset_index()
    df_both_weeks=df_both_weeks[df_both_weeks['week_end_date']==6]
    
    df=df[df['location_id'].isin(df_both_weeks['location_id'])]
    print(len(df['location_id'].unique()))
    df_base=df[df['week_end_date'].isin(base_week)].groupby(["location_id"])['sales'].sum().to_frame().reset_index()
    df_base=df_base.rename(columns={"sales":"base_sales"})
    df_campaign=df[df['week_end_date'].isin(campaign_week)].groupby(["location_id"])['sales'].sum().to_frame().reset_index()
    df_campaign=df_campaign.rename(columns={"sales":"campaign_sales","week_end_date":"campaign_week"})
    df_pair=pd.merge(df_base,df_campaign,on="location_id",how="inner")
    base_sales_all=df_pair['base_sales'].sum()
    campaign_sales_all=df_pair['campaign_sales'].sum()
    
    df_pair['ARI']=(df_pair['campaign_sales']/df_pair['base_sales'])/(campaign_sales_all/base_sales_all)*100
    
    return df_pair

In [13]:
AIR_Base_Week=sales_base_week_list.copy()
AIR_Campaign_Week=[datetime.date(2018,2,17),datetime.date(2018,2,24),datetime.date(2018,3,3)]

In [14]:
ARI_df=ARI_calculation(df_all=sales_all,base_week=AIR_Base_Week,campaign_week=AIR_Campaign_Week)
ARI_df=ARI_df[['location_id','ARI']]
'''
ARI_df_1=ARI_calculation(df_all=sales_all,base_week=base1_week,campaign_week=campaign1_week)
ARI_df_1=ARI_df_1.rename(columns={"ARI":"ARI_P1"})[['location_id','ARI_P1']]
ARI_df_2=ARI_calculation(df_all=sales_all,base_week=base2_week,campaign_week=campaign2_week)
ARI_df_2=ARI_df_2.rename(columns={"ARI":"ARI_P2"})[['location_id','ARI_P2']]
'''

1412


'\nARI_df_1=ARI_calculation(df_all=sales_all,base_week=base1_week,campaign_week=campaign1_week)\nARI_df_1=ARI_df_1.rename(columns={"ARI":"ARI_P1"})[[\'location_id\',\'ARI_P1\']]\nARI_df_2=ARI_calculation(df_all=sales_all,base_week=base2_week,campaign_week=campaign2_week)\nARI_df_2=ARI_df_2.rename(columns={"ARI":"ARI_P2"})[[\'location_id\',\'ARI_P2\']]\n'

In [15]:
Df_parameters=pd.merge(Df_parameters,ARI_df,on="location_id",how="left")
# Df_parameters=pd.merge(Df_parameters,ARI_df_2,on="location_id",how="left")

In [16]:
Df_parameters.head(2)

,location_id,52_week_sales,week_count,DMA,Jan_3_weeks_sales,ARI
0,1,2243400.62,26,"COLUMBUS, OH",202241.46,118.509320
1,1001,1123476.33,26,CHATTANOOGA,95384.23,125.956767


In [17]:
# 4. BDI (with female 25-54)

SP_Household=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/ZipcodePopulation_FromSP.csv",dtype=str)
SP_Household['ZIP_CODE']=SP_Household['ZIP_CODE'].apply(lambda x: x.zfill(5))
SP_Household=SP_Household.rename(columns={"ZIP_CODE":"zip_cd"})[['zip_cd','HH15']]
SP_Household['HH15']=SP_Household['HH15'].astype(float)

Zip_DMA=pd.read_csv("/home/jian/BiglotsCode/OtherInput/zipdmamapping.csv",dtype=str)
Zip_DMA['zip_cd']=Zip_DMA['zip_cd'].apply(lambda x:x.zfill(5))
Zip_DMA_HH=pd.merge(Zip_DMA,SP_Household,on="zip_cd",how="left")
Zip_DMA_HH=Zip_DMA_HH.fillna(0)
DMA_HH=Zip_DMA_HH.groupby(["DMA"])['HH15'].sum().to_frame().reset_index()

In [56]:
female_2554=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset.csv")
female_2554['Female_25_to_54']=female_2554[['Estimate; Female: - 25 to 29 years',
                                            'Estimate; Female: - 30 to 34 years',
                                            'Estimate; Female: - 35 to 39 years',
                                            'Estimate; Female: - 40 to 44 years',
                                            'Estimate; Female: - 45 to 49 years',
                                            'Estimate; Female: - 50 to 54 years']].sum(axis=1)
'''
female_2554=female_2554[['ZIP_CODE','Female_25_to_54']].rename(columns={"ZIP_CODE":"zip_cd"})
female_2554['zip_cd']=female_2554['zip_cd'].apply(lambda x: str(x).zfill(5))

female_2554_DMA=pd.merge(Zip_DMA,female_2554,on="zip_cd",how="left")
female_2554_DMA=female_2554_DMA.groupby(['DMA'])['Female_25_to_54'].sum().to_frame().reset_index()
female_2554_DMA.head(2)
'''

'\nfemale_2554=female_2554[[\'ZIP_CODE\',\'Female_25_to_54\']].rename(columns={"ZIP_CODE":"zip_cd"})\nfemale_2554[\'zip_cd\']=female_2554[\'zip_cd\'].apply(lambda x: str(x).zfill(5))\n\nfemale_2554_DMA=pd.merge(Zip_DMA,female_2554,on="zip_cd",how="left")\nfemale_2554_DMA=female_2554_DMA.groupby([\'DMA\'])[\'Female_25_to_54\'].sum().to_frame().reset_index()\nfemale_2554_DMA.head(2)\n'

In [58]:
female_2554[['Estimate; Female: - 25 to 29 years',
                                            'Estimate; Female: - 30 to 34 years',
                                            'Estimate; Female: - 35 to 39 years',
                                            'Estimate; Female: - 40 to 44 years',
                                            'Estimate; Female: - 45 to 49 years',
                                            'Estimate; Female: - 50 to 54 years','Female_25_to_54']].head(5)

,Estimate; Female: - 25 to 29 years,Estimate; Female: - 30 to 34 years,Estimate; Female: - 35 to 39 years,Estimate; Female: - 40 to 44 years,Estimate; Female: - 45 to 49 years,Estimate; Female: - 50 to 54 years,Female_25_to_54
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,554.0,619.0,518.0,639.0,653.0,621.0,3604.0
2,878.0,615.0,463.0,605.0,525.0,754.0,3840.0
3,9.0,0.0,0.0,0.0,0.0,0.0,9.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
Jan_Sales_for_BDI=sales_all[sales_all['week_end_date'].isin(sales_base_week_list)]
Jan_Sales_for_BDI=Jan_Sales_for_BDI[Jan_Sales_for_BDI['sales']>0]
Jan_Sales_for_BDI=Jan_Sales_for_BDI.drop_duplicates()
Jan_Sales_for_BDI=Jan_Sales_for_BDI.groupby(["location_id"])['week_end_date'].count().to_frame().reset_index()
Jan_Sales_for_BDI=Jan_Sales_for_BDI[Jan_Sales_for_BDI['week_end_date']>0] #the same as ==3


Jan_Sales_for_BDI=sales_all[sales_all['location_id'].isin(Jan_Sales_for_BDI['location_id'])]
Jan_Sales_for_BDI=Jan_Sales_for_BDI[Jan_Sales_for_BDI['week_end_date'].isin(sales_base_week_list)]

In [19]:
stores_DMA=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",sheetname="all store DMA",dtype=str)
stores_DMA=stores_DMA[['location_id','DMA']].drop_duplicates()
Jan_Sales_for_BDI=pd.merge(Jan_Sales_for_BDI,stores_DMA,on="location_id",how="left")
DMA_Jan_Sales=Jan_Sales_for_BDI.groupby(["DMA"])['sales'].sum().to_frame().reset_index()

In [20]:
DMA_Jan_Sales_HH=pd.merge(DMA_Jan_Sales,DMA_HH,on="DMA",how="left")

In [21]:
Jan_Store_Count_by_DMA=Jan_Sales_for_BDI[['location_id','DMA']].drop_duplicates()
Jan_Store_Count_by_DMA=Jan_Store_Count_by_DMA.groupby(["DMA"])['location_id'].count().to_frame().reset_index()
Jan_Store_Count_by_DMA=Jan_Store_Count_by_DMA.rename(columns={"location_id":"store_count"})

In [22]:
DMA_Jan_Sales_HH=pd.merge(DMA_Jan_Sales_HH,Jan_Store_Count_by_DMA,on="DMA",how="left")

In [23]:
DMA_Jan_Sales_HH['HH_per_Store']=DMA_Jan_Sales_HH['HH15']/DMA_Jan_Sales_HH['store_count']
sales_all_DMA_Jan_Sales_HH=DMA_Jan_Sales_HH['sales'].sum()
HH_all_DMA_Jan_Sales_HH=DMA_Jan_Sales_HH['HH15'].sum()

DMA_Jan_Sales_HH['BDI']=(DMA_Jan_Sales_HH['sales']/DMA_Jan_Sales_HH['HH15'])/(sales_all_DMA_Jan_Sales_HH/HH_all_DMA_Jan_Sales_HH)*100

df_BDI_by_DMA=DMA_Jan_Sales_HH[['DMA','BDI']]

Df_parameters=pd.merge(Df_parameters,df_BDI_by_DMA,on="DMA",how="left")

In [24]:
# 5.0 HH per store (HH per DMA devided by store counts)
df_HH_per_store_by_DMA=DMA_Jan_Sales_HH[['DMA','HH_per_Store']]
Df_parameters=pd.merge(Df_parameters,df_HH_per_store_by_DMA,on="DMA",how="left")

In [25]:
# 5.1 Female 25-54 per store
female_2554=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset.csv")
female_2554['Female_25_to_54']=female_2554[['Estimate; Female: - 25 to 29 years',
                                            'Estimate; Female: - 30 to 34 years',
                                            'Estimate; Female: - 35 to 39 years',
                                            'Estimate; Female: - 40 to 44 years',
                                            'Estimate; Female: - 45 to 49 years',
                                            'Estimate; Female: - 50 to 54 years',]].sum(axis=1)
female_2554=female_2554[['ZIP_CODE','Female_25_to_54']].rename(columns={"ZIP_CODE":"zip_cd"})
female_2554['zip_cd']=female_2554['zip_cd'].apply(lambda x: str(x).zfill(5))
female_2554_DMA=pd.merge(Zip_DMA,female_2554,on="zip_cd",how="left")
female_2554_DMA=female_2554_DMA.groupby(['DMA'])['Female_25_to_54'].sum().to_frame().reset_index()
female_2554_DMA=pd.merge(DMA_Jan_Sales_HH,female_2554_DMA,on="DMA",how="left")
female_2554_DMA['Female_25_54_per_Store']=female_2554_DMA['Female_25_to_54']/female_2554_DMA['store_count']
female_2554_DMA=female_2554_DMA[['DMA','Female_25_54_per_Store']]

In [26]:
Df_parameters=pd.merge(Df_parameters,female_2554_DMA,on="DMA",how="left")

In [27]:
# 6 and 7 Labels (Use 2017 calander year labels)
labels=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/Test_vs_Control/Other_input/BL_Store Sales and Ratio Groups 2017 Year_JL_20180110.xlsx",sheetname="store list with group Nov. 15",skiprows=1,dtype=str)
labels=labels[['location_id','state_nm','city_nm','Sales Group','Ratio Group']]
Df_parameters=pd.merge(Df_parameters,labels,on="location_id",how="left")


In [28]:
Df_parameters=Df_parameters[['location_id','state_nm','city_nm','DMA','52_week_sales','Jan_3_weeks_sales','HH_per_Store',
                            'Female_25_54_per_Store','ARI','BDI','Sales Group','Ratio Group']]

In [29]:
Df_parameters.to_csv("/home/jian/Projects/Big_Lots/Q1_Post/Test_vs_Control/parameters for match stores"+today_str+".csv")

In [30]:
Df_parameters.head(2)

,location_id,state_nm,city_nm,DMA,52_week_sales,Jan_3_weeks_sales,HH_per_Store,Female_25_54_per_Store,ARI,BDI,Sales Group,Ratio Group
0,1,OH,COLUMBUS,"COLUMBUS, OH",2243400.62,202241.46,47791.1600,25206.147155,118.509320,167.766311,H,H
1,1001,TN,McMINNVILLE,CHATTANOOGA,1123476.33,95384.23,50243.3875,25042.647974,125.956767,137.390414,L,L


# Match T and C

In [31]:
# 60 TV
# 40 Digital

DMA_Type=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",sheetname="DMA List",dtype=str)
DMA_Type.head(2)

Df_parameters['Campaign_Type']=np.where(Df_parameters['DMA'].isin(DMA_Type['TV+Digital']),"TV_Digital",
                                        np.where(Df_parameters['DMA'].isin(DMA_Type['Digital Only']),"Digital Only","Balance")
                                                )

In [53]:
df_Control.shape

(824, 13)

In [32]:
df_Test=Df_parameters[Df_parameters['Campaign_Type'].isin(["TV_Digital","Digital Only"])]
df_Control=Df_parameters[~Df_parameters['Campaign_Type'].isin(["TV_Digital","Digital Only"])]


df_Test_TV=df_Test[df_Test['Campaign_Type']=="TV_Digital"]
df_Test_Digital=df_Test[df_Test['Campaign_Type']=="Digital Only"]

In [33]:
df_Control_temp=df_Control[['location_id','Campaign_Type','52_week_sales','Jan_3_weeks_sales','HH_per_Store','Female_25_54_per_Store','ARI','BDI','Sales Group','Ratio Group']]
df_Control_temp.columns=["C_"+x for x in ['location_id','Campaign_Type','52_week_sales','Jan_3_weeks_sales','HH_per_Store','Female_25_54_per_Store','ARI','BDI','Sales Group','Ratio Group']]
df_Control_temp['temp_key']=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
len(Df_parameters['DMA'].unique())

186

In [35]:
group_label_diff_score=0.1

df_match_all=pd.DataFrame()
k_loop=0
logging.info(str(datetime.datetime.now())+ " : Start to do the match")
for location_tv in df_Test["location_id"].unique().tolist():
    df_t=df_Test[df_Test['location_id']==location_tv].reset_index()
    df_t=df_t[['location_id','Campaign_Type','52_week_sales','Jan_3_weeks_sales','HH_per_Store','Female_25_54_per_Store','ARI','BDI','Sales Group','Ratio Group']]
    df_t.columns=["T_"+x for x in ['location_id','Campaign_Type','52_week_sales','Jan_3_weeks_sales','HH_per_Store','Female_25_54_per_Store','ARI','BDI','Sales Group','Ratio Group']]
    df_t['temp_key']=1
    df_match=pd.DataFrame()
    df_match=pd.merge(df_t,df_Control_temp,on="temp_key",how="left")
    
    df_match['diff_52_week_sales']=np.nan
    df_match['diff_Jan_3_weeks_sales']=np.nan
    df_match['diff_HH_per_Store']=np.nan
    df_match['diff_ARI']=np.nan
    df_match['diff_BDI']=np.nan
    df_match['diff_sales_lable']=np.nan
    df_match['diff_ratio_lable']=np.nan
    diff_52_week_sales_list=[np.nan]*len(df_Control_temp)
    diff_Jan_3_weeks_sales_list=[np.nan]*len(df_Control_temp)
    diff_HH_per_Store_list=[np.nan]*len(df_Control_temp)
    diff_Female_per_Store_list=[np.nan]*len(df_Control_temp)
    diff_ARI_list=[np.nan]*len(df_Control_temp)
    diff_BDI_list=[np.nan]*len(df_Control_temp)
    diff_sales_lable_list=[np.nan]*len(df_Control_temp)
    diff_ratio_lable_list=[np.nan]*len(df_Control_temp)
    
    
    
    for i in range(len(df_Control_temp)):
        
        diff_52_week_sales_list[i]=(df_match['T_52_week_sales'][i]-df_match['C_52_week_sales'][i])/df_match['C_52_week_sales'][i]
        diff_Jan_3_weeks_sales_list[i]=(df_match['T_Jan_3_weeks_sales'][i]-df_match['C_Jan_3_weeks_sales'][i])/df_match['C_Jan_3_weeks_sales'][i]
        diff_HH_per_Store_list[i]=(df_match['T_HH_per_Store'][i]-df_match['C_HH_per_Store'][i])/df_match['C_HH_per_Store'][i]
        diff_ARI_list[i]=(df_match['T_ARI'][i]-df_match['C_ARI'][i])/df_match['C_ARI'][i]
        diff_BDI_list[i]=(df_match['T_BDI'][i]-df_match['C_BDI'][i])/df_match['C_BDI'][i]
        diff_Female_per_Store_list[i]=(df_match['T_Female_25_54_per_Store'][i]-df_match['C_Female_25_54_per_Store'][i])/df_match['C_Female_25_54_per_Store'][i]
        
        if df_match['T_Sales Group'][i]==df_match['C_Sales Group'][i]:
            diff_sales_lable_list[i]=0
        else:
            diff_sales_lable_list[i]=group_label_diff_score
            
        if df_match['T_Ratio Group'][i]==df_match['C_Ratio Group'][i]:
            diff_ratio_lable_list[i]=0
        else:
            diff_ratio_lable_list[i]=group_label_diff_score
            
    df_match['diff_52_week_sales']=diff_52_week_sales_list
    df_match['diff_Jan_3_weeks_sales']=diff_Jan_3_weeks_sales_list
    df_match['diff_HH_per_Store']=diff_HH_per_Store_list
    df_match['diff_Female_per_Store']=diff_Female_per_Store_list
    df_match['diff_ARI']=diff_ARI_list
    df_match['diff_BDI']=diff_BDI_list
    df_match['diff_sales_lable']=diff_sales_lable_list
    df_match['diff_ratio_lable']=diff_ratio_lable_list

    
    df_match_all=df_match_all.append(df_match)
    
    k_loop=k_loop+1
    if k_loop%100==1:
        logging.info(str(k_loop)+": "+str(datetime.datetime.now()))
        print(k_loop,datetime.datetime.now())
    if k_loop%100==20:
        logging.info(str(k_loop)+": "+str(datetime.datetime.now()))
        print(k_loop,datetime.datetime.now())

logging.info("Finished matching: "+str(datetime.datetime.now()))   


1 2018-06-25 10:19:21.071903
20 2018-06-25 10:19:27.211602
101 2018-06-25 10:19:54.738531
120 2018-06-25 10:20:01.053694
201 2018-06-25 10:20:30.169787
220 2018-06-25 10:20:37.852254
301 2018-06-25 10:21:12.189921
320 2018-06-25 10:21:20.206287
401 2018-06-25 10:21:57.383138
420 2018-06-25 10:22:06.576197
501 2018-06-25 10:22:46.536939
520 2018-06-25 10:22:56.756741


In [45]:
df_match_all['score']=df_match_all['diff_Jan_3_weeks_sales'].apply(lambda x: abs(x))+\
                        df_match_all['diff_Female_per_Store'].apply(lambda x: abs(x))+\
                        df_match_all['diff_ARI'].apply(lambda x: abs(x))+\
                        df_match_all['diff_BDI'].apply(lambda x: abs(x))+\
                        df_match_all['diff_sales_lable'].apply(lambda x: abs(x))+\
                        df_match_all['diff_ratio_lable'].apply(lambda x: abs(x))
                        
# df_match_all['diff_52_week_sales'].apply(lambda x: abs(x))+\                    

In [46]:
df_match_all=df_match_all.sort_values('score')
# df_match_all.to_csv("/home/jian/Projects/Big_Lots/Q1_Post/Test_vs_Control/match_overall.csv",index=False)

In [47]:
df_match_all.head(5)

,T_location_id,T_Campaign_Type,T_52_week_sales,T_Jan_3_weeks_sales,T_HH_per_Store,T_Female_25_54_per_Store,T_ARI,T_BDI,T_Sales Group,T_Ratio Group,...,C_Ratio Group,diff_52_week_sales,diff_Jan_3_weeks_sales,diff_HH_per_Store,diff_ARI,diff_BDI,diff_sales_lable,diff_ratio_lable,diff_Female_per_Store,score
633,1595,TV_Digital,2320126.55,215055.08,72403.543750,40034.437500,101.549852,118.941269,H,M,...,M,0.077317,-0.017307,-0.158908,-0.000142,-0.011662,0.0,0.0,0.006674,0.035785
384,1829,TV_Digital,1424071.70,122146.77,55080.561538,27578.384615,110.365935,130.242075,L,M,...,M,0.090868,-0.014721,0.044899,0.008516,-0.006985,0.0,0.0,0.020149,0.050371
227,5268,TV_Digital,1845178.26,182547.87,53470.063158,27296.421053,109.134036,133.014649,M,M,...,M,-0.058547,0.013562,0.014347,0.015409,0.014154,0.0,0.0,0.009719,0.052844
349,5141,Digital Only,1378789.84,128624.44,74864.340000,37362.400000,106.682102,91.784260,L,L,...,L,-0.019396,0.014631,-0.038545,0.002096,-0.016461,0.0,0.0,0.025022,0.058210
632,1799,TV_Digital,1463031.91,141282.63,60927.707692,30820.153846,97.659759,106.698092,M,M,...,M,-0.065973,0.001717,-0.071258,0.018577,0.034053,0.0,0.0,0.005610,0.059957


In [48]:
df_match_all_copy=df_match_all.copy()
df=df_match_all_copy.head(1)
df['T_location_id'].unique().tolist()[0]

'1595'

In [49]:
df_match_all_copy=df_match_all.copy()
selected_T=[]
selected_C=[]
df_selected_match=pd.DataFrame()
i=0
while i <100:
    df_match_all_copy=df_match_all_copy[(~df_match_all_copy['T_location_id'].isin(selected_T)) & (~df_match_all_copy['C_location_id'].isin(selected_C))]
    df=df_match_all_copy.head(1)
    df_selected_match=df_selected_match.append(df)

    selected_T=selected_T+[df['T_location_id'].unique().tolist()[0]]
    selected_C=selected_T+[df['C_location_id'].unique().tolist()[0]]
    i=i+1
    
    
    
    
    

In [50]:
df_selected_match=df_selected_match.sort_values(["T_Campaign_Type","score"])

In [51]:
df_selected_match.to_csv("/home/jian/Projects/Big_Lots/Q1_Post/Test_vs_Control/selectet_top_100_match_pairs"+today_str+".csv")

